![alt text](./Cerny_logo_1.jpg)

##### Analysis of Cerny ventilation recordings

## Analysis of mechanically ventilated cases `AL000601-AL000665`

### Importing the necessary libraries and setting options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
import sklearn as sk

import os
import sys
import re
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
# pd.set_option('mode.chained_assignment', None) 

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("IPython version: {}".format(IPython.__version__))
print("scikit-learn version: {}".format(sk.__version__))

### List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'fabian'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = '/Users/guszti/ventilation_fabian'

# Directory on external drive to read the ventilation data from
DIR_READ = '/Volumes/%s/Fabian/fabian_data' % DRIVE

DIR_WRITE = '%s/%s/%s' % (CWD, 'Analyses', 'analysis_ventilated_601_665')
if not os.path.isdir(DIR_WRITE):
    os.makedirs(DIR_WRITE)

# Images and raw data will be written on an external hard drive
if not os.path.isdir('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)):
    os.makedirs('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC))
DATA_DUMP = '/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)

In [ ]:
os.chdir(CWD)
os.getcwd()

In [ ]:
DIR_READ, DIR_WRITE, DATA_DUMP

### Import ventilator and clinical data from pickle archives

In [ ]:
# Import ventilator parameters, settings and alarms

with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_measurements_ventilated_601_665'), 'rb') as handle:
    data_pars_measurements_ventilated = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_settings_ventilated_601_665'), 'rb') as handle:
    data_pars_settings_ventilated = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_alarms_ventilated_601_665'), 'rb') as handle:
    data_pars_alarms_ventilated = pickle.load(handle)


# Import DataFrame with ventilation modes

with open('%s/%s.pickle' % (DATA_DUMP, 'vent_modes_ventilated_601_665'), 'rb') as handle:
    vent_modes_ventilated = pickle.load(handle)


# Import clinical data

with open('%s/%s.pickle' % (DATA_DUMP, 'clin_df_601_900'), 'rb') as handle:
    clin_df = pickle.load(handle)

In [ ]:
len(data_pars_measurements_ventilated)

In [ ]:
cases = sorted(data_pars_measurements_ventilated.keys())

### Import table for interpreting ventilator parameters

In [ ]:
par_key_table = pd.read_excel('Fabian_parameters.xlsx')
par_key_table;

## Statistics on clinical details of these cases

In [ ]:
len(clin_df)

In [ ]:
clin_df_ventilation = clin_df.loc[vent_modes_ventilated.index]

In [ ]:
clin_df_ventilation.head(2)

In [ ]:
clin_df_ventilation.info()

In [ ]:
clin_df_ventilation_stats = round(clin_df_ventilation.describe(), 2)
clin_df_ventilation_stats

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_data_ventilated_601_665.xlsx'))
clin_df_ventilation.to_excel(writer, 'ventilated_cases')
clin_df_ventilation_stats.to_excel(writer, 'stats')
writer.save()

In [ ]:
fig, ax = plt.subplots(figsize = (6, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['gestational age', 'corrected gestational age']
conversion_number = 1e9 * 60 * 60 * 24 * 7 # convert nanoseconds to weeks

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_ventilation['Gestational Age (weeks)'] ,
             clin_df_ventilation['Corrected gestational Age (weeks)']] ,
            whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
            whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_ylabel('weeks', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'ventilated_gest_age_601_665', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

In [ ]:
fig, ax = plt.subplots(figsize = (6, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['birth weight', 'actual weight']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_ventilation['Birth Weight'], clin_df_ventilation['Weight']],
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 5500)
ax.set_ylabel('grams', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'ventilated_weight_601_665', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['recording duration']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot(clin_df_ventilation['Duration'] / (60 * 1E+9), 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylabel('minutes', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'ventilation_duration_601_665', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

## How many cases with the different ventilation modes

### Which recordings had more than one ventilation modes

In [ ]:
vent_modes_ventilated.head()

In [ ]:
ventilated = vent_modes_ventilated.reindex(['SIMV', 'SIMVPSV', 'SIPPV', 'IPPV', ], axis = 1)
multiple_mode = [case for case in cases if (ventilated.loc[case] >0).sum() > 1]

In [ ]:
len(multiple_mode)

In [ ]:
vent_modes_ventilated['multiple_mode'] = np.where(vent_modes_ventilated.index.isin(multiple_mode), 'Yes', 'No')

In [ ]:
vent_modes_ventilated.loc[multiple_mode]

In [ ]:
def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height), ha='center', va='bottom', size = 14)

### Considering all modes used during recordings

In [ ]:
print('SIMV:', sum(vent_modes_ventilated['SIMV'] > 0))
print('SIPPV:', sum(vent_modes_ventilated['SIPPV'] > 0))
print('IPPV:', sum(vent_modes_ventilated['IPPV'] > 0))
print('VG_on:', sum(vent_modes_ventilated['VG_on'] > 0))
print('total', len(vent_modes_ventilated))

In [ ]:
dpi = 300
filetype = 'jpg'
labels = ['SIMV', 'SIPPV', 'IPPV']
xticks = np.arange(len(labels))
width = 0.6

fig, ax = plt.subplots(figsize = [4,4])
rects = plt.bar(xticks, [sum(vent_modes_ventilated['SIMV'] > 0), sum(vent_modes_ventilated['SIPPV'] > 0), 
                         sum(vent_modes_ventilated['IPPV'] > 0)], 
                        width=width, color='black', alpha  = 0.75, align = 'center')

ax.set_xlabel('ventilation mode', size = 14)
ax.set_xticks(xticks)
ax.set_xticklabels(labels, size = 14, rotation = 0)
ax.set_ylabel('number of cases', size = 14)
ax.set_title('Ventilation mode used')
ax.set_ylim(0, 30)
ax.grid(True)

autolabel(rects)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vent_modes_ventilated_601_665', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

### Considering only the predominant ventilation modes

In [ ]:
vent_modes_ventilated['dominant_mode'] = \
    vent_modes_ventilated[['SIMV', 'SIPPV', 'IPPV']].idxmax(axis = 1)

In [ ]:
vent_modes_ventilated.head()

In [ ]:
vent_modes_ventilated['dominant_mode'].value_counts()

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = vent_modes_ventilated['dominant_mode'].value_counts().index
xticks = np.arange(len(vent_modes_ventilated['dominant_mode'].value_counts()))
fig, ax = plt.subplots(figsize = [4,4])

rects = plt.bar(xticks, vent_modes_ventilated['dominant_mode'].value_counts(), color='black',
                                            width = 0.7, alpha  = 0.75, )
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels, size = 14, rotation = 0)
ax.set_xlabel('ventilation mode', size = 14)
ax.set_ylabel('number of cases', size = 14)
ax.set_title('Predominant ventilation mode')
ax.set_ylim(0, 30)
ax.grid(True)

autolabel(rects)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vent_modes_ventilated_dominant_mode_601_665', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

### Considering only the recordings with a single ventilator mode

In [ ]:
sum(vent_modes_ventilated['multiple_mode'] == 'No')

In [ ]:
only_mode = vent_modes_ventilated[vent_modes_ventilated['multiple_mode'] == 'No']['dominant_mode'].value_counts()
only_mode

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = only_mode.index
xticks = np.arange(len(only_mode))
fig, ax = plt.subplots(figsize = [4,4])

rects = plt.bar(xticks, only_mode, color='black', width = 0.7, alpha  = 0.75)
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels, size = 14, rotation = 0)
ax.set_xlabel('ventilation mode', size = 14)
ax.set_ylabel('number of cases', size = 14)
ax.set_title('Cases with a single ventilator mode')
ax.set_ylim(0, 25)
ax.grid(True)

autolabel(rects)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vent_modes_ventilated_only_mode_601_665', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

### How many recordings had VG ventilation and for how long

In [ ]:
# How many nan values
sum(vent_modes_ventilated['VG_on'].isnull())

In [ ]:
# has some VG ventilationa
sum(vent_modes_ventilated['VG_on'] > 0)

In [ ]:
# has no VG ventilation
len(vent_modes_ventilated) - sum(vent_modes_ventilated['VG_on'] > 0)

In [ ]:
has_VG = vent_modes_ventilated[vent_modes_ventilated['VG_on'] > 0]

In [ ]:
# VG was on throughout the whole recording 
only_VG = has_VG[has_VG['VG_on'] == has_VG['total']]
len(only_VG)

In [ ]:
only_VG['dominant_mode'].value_counts()

#### Save extended file about ventilation modes to pickle archive

In [ ]:
with open('%s/%s.pickle' % (DATA_DUMP, 'vent_modes_ventilated_601_665_plus'), 'wb') as handle:
    pickle.dump(vent_modes_ventilated, handle, protocol=pickle.HIGHEST_PROTOCOL)